# Reconstruction (without truncations)
_Author: Lucas Z. Brito_

This notebook reconstructs the Haldane Shastry Hamiltonian. 

### Reconstruction and model parameters

In [17]:
n = 15 # Chain size 
atol = 1e-2 # Absolute error for sanity checks 
output_dir = "../data/hs_data/"

"../data/hs_data/"

In [18]:
using JLD
using DataFrames
using CSV

### Reconstruction

In [19]:
include("./src/hs_tools.jl")

# Make wavefunction and correlation matrix and save to a directory to avoid 
# running calculation again. 
# make_wf_cm(n, output_dir) # Can take roughly a minute for n > 10. 

ft_mat

In [20]:
cm_wf = load(output_dir * "cm_wf_n=$(n).jld")
k = cm_wf["wf"]
cm = cm_wf["cm"]

# Haldane shastry coefficients
hs = normalize(hs_coeffs(n))
hs

7-element Vector{Float64}:
 0.9545164635108843
 0.24941044899105871
 0.11942737137332071
 0.07471268217145025
 0.05501485601695768
 0.045617196677552305
 0.04171694973359136

In [21]:
# Make correlation matrix
# cm = make_corr_mat(n, k)

printstyled("Correlation matrix: ", bold=true); flush(stdout)
display(cm)

# Diagonalize correlation matrix
eig = eigen(cm)

printstyled("Eigenvectors:", bold=true); flush(stdout)
display(eig.vectors)
printstyled("Eigenvalues:", bold=true); flush(stdout)
display(eig.values)

Correlation matrix: 

7×7 Matrix{Float64}:
  0.000148244  -0.00105604    0.00133091  …  -0.00027468   9.73776e-5
 -0.00105604    0.00783289   -0.0106379       0.00201063  -0.000652366
  0.00133091   -0.0106379     0.0169953      -0.00357073   0.00114727
 -0.000765064   0.00622744   -0.0122958       0.0063077   -0.00191625
  0.000519251  -0.00372461    0.00703097     -0.0110755    0.00336124
 -0.00027468    0.00201063   -0.00357073  …   0.0127606   -0.00615803
  9.73776e-5   -0.000652366   0.00114727     -0.00615803   0.00412077

Eigenvectors:

7×7 Matrix{Float64}:
  0.126542  0.971031     0.162166   …   0.0461879   0.0483046   0.0365501
 -0.308779  0.220943    -0.519117      -0.334357   -0.390117   -0.283295
 -0.389029  0.0826679   -0.474557       0.25098     0.556503    0.482074
 -0.416635  0.0351007   -0.184237       0.455535   -0.0959115  -0.529129
 -0.428796  0.0141462    0.0914333     -0.403587   -0.3968      0.514765
 -0.434598  0.00414906   0.393107   …  -0.483078    0.545509   -0.351977
 -0.437006  0.0          0.531205       0.468319   -0.267488    0.131012

Eigenvalues:

7-element Vector{Float64}:
 -2.1776887758105045e-17
  2.374063944564353e-16
  0.00044912975970302145
  0.0019644949770810873
  0.0068024889751717365
  0.017622502379275536
  0.047270374189586906

In [22]:
# Check that conserved quantity and Hamiltonian are in nullspace of CM
nullsp = eig.vectors[:, 1:2]

h_proj, h_is_in_nsp = in_subspace(nullsp, hs, atol)
# in_subspace([eig.vectors[:, 1]], hs, atol)

conserved_quant = normalize(ones(trunc(Int, n / 2)))
stot_proj, stot_is_in_nsp = in_subspace(nullsp, conserved_quant, atol)

# Project HS coefficient onto nullspace; this will be our "reconstructed 
# Hamiltonian."
coeffs_recon = nullsp * inv(nullsp' * nullsp) * nullsp' * hs

if h_is_in_nsp
	printstyled("Hamiltonian is in nullspace ✓", bold=true, color=:green)
else 
	printstyled("Hamiltonian is not in nullspace ×", bold=true, color=:red)
end 

println()

if stot_is_in_nsp
	printstyled("S_tot is in nullspace ✓", bold=true, color=:green)
else 
	printstyled("S_tot is not in nullspace ×", bold=true, color=:red)
end 

println()
println("(atol = $atol)")

Hamiltonian is in nullspace ✓
S_tot is in nullspace ✓
(atol = 0.01)


In [23]:
printstyled("Projections onto nullspace:\n", bold=true)
print("H:\t")
print(h_proj)
println()
print("S_tot:\t")
print(stot_proj)
println()

Projections onto nullspace:
H:	1.0000000000000002
S_tot:	1.0000000000000002


In [24]:
# Save data 
df = DataFrame(r=1:(trunc(Int, n / 2)), 
				n=n,
				coeffs_anal=hs, 
				coeffs_recon=real.(coeffs_recon))
fn = CSV.write(output_dir * "hs_recon_n=$n.csv", df)
printstyled("Written reconstruction data to $fn.", bold=true, color=:green)

UndefVarError: UndefVarError: fn not defined